In [1]:
# pip install pandas

In [2]:
# pip install gensim

In [3]:
# pip install scipy

In [4]:
# pip install pyLDAvis

In [17]:
import pandas as pd
import numpy as np
import glob
from nltk.corpus import stopwords


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords

# vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [19]:
df = pd.read_parquet('process_2.parquet')
df

,data_process,data_display,Keyword,Abstract,processed_documents_s,vector,x,y
0,two archeological sites discovered recently is...,id: Gemstone_1 Title: Pre-Columbian Gems and O...,Gemstone,Two archeological sites that were discovered r...,"[two, archeological, sites, discovered, recent...","[-0.0057970188, 0.055725425, -0.05079549, 0.02...",-0.732679,2.172171
1,figure 1 five 10 jadeite samples levoketchpel ...,id: Gemstone_2 Title: Ornamental Jadeites from...,Gemstone,Figure 1. Five of the 10 jadeite samples from ...,"[figure, 1, five, 10, jadeite, samples, levoke...","[0.04978066, -0.031103676, -0.022980962, 0.039...",2.526106,5.257230
2,figure 1 single tourmaline fiber contains thre...,id: Gemstone_3 Title: Colored Stones and Pearl...,Gemstone,Figure 1. A single tourmaline fiber contains t...,"[figure, 1, single, tourmaline, fiber, contain...","[-0.010867849, -0.021513185, -0.045135252, 0.0...",2.571761,2.733680
4,figure 1 paisley earrings ca 1885 gilt silver ...,id: Gemstone_5 Title: Exhibition Review: Ornam...,Gemstone,Figure 1. Paisley earrings ca. 1885. Gilt silv...,"[figure, 1, paisley, earrings, ca, 1885, gilt,...","[0.040627435, 0.021101465, -0.02011571, 0.0733...",-2.286749,1.917985
5,noël adams hardcover 432 pp illus publ morgan ...,id: Gemstone_6 Title: Books: Bright Lights in ...,Gemstone,"By Noël Adams, hardcover, 432 pp., illus., pub...","[noël, adams, hardcover, 432, pp, illus, publ,...","[0.04624943, 0.077438615, -0.010378159, 0.0300...",-2.541490,2.226159
...,...,...,...,...,...,...,...,...
110,800 shell remains late ottoman british mandate...,id: nacre_jewelry_155 Title: Button waste and ...,None,Over 800 shell remains from the late Ottoman a...,"[800, shell, remains, late, ottoman, british, ...","[-0.03222377, 0.0019210138, 0.008421725, 0.039...",-1.965013,2.024704
111,socalled new media technologies often referred...,id: nacre_jewelry_156 Title: The New Media Tec...,None,The so-called new media technologies - often r...,"[socalled, new, media, technologies, often, re...","[0.037681274, -0.07877747, -0.04059247, -0.066...",-3.200164,0.147219
113,past 15 years combination market forces enviro...,id: nacre_jewelry_158 Title: From Single Sourc...,None,Over the past 15 years a combination of market...,"[past, 15, years, combination, market, forces,...","[0.026291138, 0.029178105, -0.043042358, 0.007...",-2.606993,4.774700
114,cultured pearls human creations formed inserti...,"id: nacre_jewelry_161 Title: Yes, it turns: Ex...",None,Cultured pearls are human creations formed by ...,"[cultured, pearls, human, creations, formed, i...","[0.021074304, -0.04415614, -0.007122643, -0.00...",-1.630545,6.786059


In [21]:
stopwords = stopwords.words("english")

In [23]:
# print (stopwords)

In [25]:
data = df['Abstract']
data[0][0:90]

0    Two archeological sites that were discovered r...
0    Natural pearls are formed when sand or parasit...
Name: Abstract, dtype: object

In [27]:
# Filter for rows with non-null values in both title and abstract
df_filtered = df[~df['Abstract'].isnull()]
data = df_filtered['Abstract']

In [29]:
# Load spaCy model (disable unnecessary components for efficiency)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out

# Lemmatize the combined titles and abstracts
lemmatized_texts = lemmatization(data.tolist())

# Print the first 90 characters of the first lemmatized text
print(lemmatized_texts[0][0:90])

archeological site discover recently island appear flourish lapidary industry excavation s


In [30]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['archeological', 'site', 'discover', 'recently', 'island', 'appear', 'flourish', 'lapidary', 'industry', 'excavation', 'site', 'date', 'about', 'ad', 'period', 'reveal', 'bead', 'pendant', 'make', 'variety']


In [31]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]
about


In [32]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=25,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [33]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
20    -0.173615 -0.315810       1        1  13.113327
14     0.389899 -0.026054       2        1  11.073592
5      0.036519 -0.343015       3        1   8.847952
13     0.314214 -0.230473       4        1   8.771004
9      0.093308 -0.109996       5        1   8.749696
22    -0.294367 -0.159212       6        1   5.550013
1     -0.096197 -0.417932       7        1   5.101472
3      0.172841 -0.223755       8        1   5.034400
10    -0.122897 -0.197033       9        1   4.620731
18     0.262116  0.047095      10        1   4.059615
12     0.422258 -0.121995      11        1   3.800921
11     0.451214  0.150427      12        1   3.758937
6     -0.294955  0.022640      13        1   3.421009
8     -0.436608 -0.060205      14        1   3.322762
16     0.257321 -0.405662      15        1   2.752858
15    -0.374406 -0.292855      16        1   2.186940
23     0.081416  0.126023      17        1   2.132980
2     -0.412812  0.178861      18        1   1.628143
19     0.251400  0.328596      19        1   1.016203
17     0.104411  0.388456      20        1   0.448336
24    -0.024638  0.409003      21        1   0.164239
0     -0.179338  0.238391      22        1   0.158704
7     -0.128116  0.394171      23        1   0.152365
4     -0.205639  0.338516      24        1   0.127173
21    -0.093329  0.281819      25        1   0.006628, topic_info=           Term         Freq        Total Category  logprob  loglift
717       pearl  4363.000000  4363.000000  Default  10.0000  10.0000
442     diamond  7022.000000  7022.000000  Default   9.0000   9.0000
1007    jewelry  5506.000000  5506.000000  Default   8.0000   8.0000
582   inclusion  4256.000000  4256.000000  Default   7.0000   7.0000
867   synthetic  3659.000000  3659.000000  Default   6.0000   6.0000
...         ...          ...          ...      ...      ...      ...
0         about     0.003054  1023.480192  Topic25  -9.7235  -3.1007
1      abundant     0.003054   123.880943  Topic25  -9.7235  -0.9891
2            ad     0.003054   103.486356  Topic25  -9.7235  -0.8092
3      amethyst     0.003054   389.747362  Topic25  -9.7235  -2.1352
4        appear     0.003054   790.277010  Topic25  -9.7235  -2.8421

[512 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.136788   about
0         2  0.188572   about
0         3  0.189549   about
0         5  0.019541   about
0         7  0.232540   about
...     ...       ...     ...
2675      3  0.122661   young
2675      6  0.110080   young
2675      7  0.072339   young
2675     14  0.691934   young
948      12  0.994057  zoning

[1277 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 15, 6, 14, 10, 23, 2, 4, 11, 19, 13, 12, 7, 9, 17, 16, 24, 3, 20, 18, 25, 1, 8, 5, 22])

In [ ]:
def print_topics(model, num_words):
    words = []
    for topic_id, topic in model.show_topics(formatted=False, num_words=num_words):
        word_list = [word for word, prob in topic]
        words.append(word_list)
    return words

# Assuming you have already defined lda_model, id2word, and corpus

# Print the top 10 words for each topic
num_words = 10
topic_words = print_topics(lda_model, num_words)

# Determine the actual number of topics
num_topics = len(topic_words)

# Create a Pandas DataFrame for better visualization
df = pd.DataFrame(topic_words, columns=[f"Word_{i+1}" for i in range(num_words)])
df.index = [f"Topic_{i}" for i in range(num_topics)]
print(df)

In [34]:
num_topics = lda_model.num_topics
words_list = []

# Extract words for each topic
for i in range(num_topics):
    topic_words = lda_model.show_topic(i, topn=10)  # Adjust topn as needed
    for word, prob in topic_words:
        words_list.append({'Topic': f'Topic_{i+1}', 'Word': word})

# Create a DataFrame
df = pd.DataFrame(words_list)

# Save to CSV
df.to_csv('topics_words_jewelry.csv', index=False)

df

,Topic,Word
0,Topic_1,strain
1,Topic_1,site
2,Topic_1,predominantly
3,Topic_1,shopping
4,Topic_1,great
...,...,...
245,Topic_25,responsible
246,Topic_25,issue
247,Topic_25,chain
248,Topic_25,practice
